# Feeding city_bikes.ipynd Latitude and Longitude Values Into Foursquare API

In [2]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
import json

In [3]:
# loading the .env file
load_dotenv()

Python-dotenv could not parse statement starting at line 7


True

In [4]:
# assigning stored api_key in .env file to vairables
api_key = os.environ.get('FOURSQUARE_API_KEY')
print(api_key)

fsq3g6XUjBbzp5TmQqvOA/ZqluKiotmdvlpoHm+9eLbvKWU=


In [5]:
# setting up base_url for foursquare
fsq_url = "https://api.foursquare.com/v3/places/search"

# Headers for authorization (api key)
fsq_headers = {
	"Accept": "application/json",
    "Authorization": api_key
}

In [6]:
# reading in the CityBikes - BikeShareToronto.csv (created in city_bikes.ipynb)
cityBikes = pd.read_csv('BikeShareToronto.csv')
cityBikes

# Pulling all bike stations in toronto, we get 717 stations in total, with their respective lat/lon's and the availibility of free bikes.

,Unnamed: 0,latitude,longitude,free_bikes
0,0,43.665269,-79.319796,18
1,1,43.671420,-79.445947,2
2,2,43.666224,-79.317693,10
3,3,43.653236,-79.376716,4
4,4,43.663722,-79.380288,23
...,...,...,...,...
712,712,43.601915,-79.499419,1
713,713,43.642852,-79.531995,1
714,714,43.650945,-79.379498,0
715,715,43.707356,-79.437231,0


### Instead of calculating Euclidean distance, comparing each latitude/longitude of bars/resturants/libraries to each latitude/longitude of each bike station, instead we're going to feed the latitudes/longitudes of each bike stop into the Foursquare API call, adjusting the radius to 1000m (1km) and setting the limit to the maximum of 50 - this will give us the total number of POI's (depending on what our search category is) within a 1k radius of each bike station.

In [8]:
# setting up lat and lon values for cityBike stations
lat = cityBikes['latitude'].values # will give a numpy array of each bike station latitude
lon = cityBikes['longitude'].values
len_cityBikes = len(cityBikes) # will give the number of bike stations - were going to use this length/value below when we iterate using a for-loop.


In [9]:
# intitialising lists for number of bars and libs and the minimum distance to the closest bar and lib within a 1000m (1km) radius of each bike stand. 
# These will ulitmately be added to to cityBikes dataFrame and used for modelling and EDA. The goal here is to rid away with joinig tables with pandas or SQL and essentially,
# perform everything uptream. This method ensure data validity and integrity - ensuring no duplicates and issues that come about with joining on keys etc..


num_of_bars = [] # number of bars within 1km of bike station
num_of_libs = [] # ||
min_bars_dist = [] # distance of closest bar to bike station in meters
min_libs_dist = [] # ||


for k in range(0, 4): # iterating through range all the way up to len of cityBikes (717 bike stations)
    ll = f'{lat[k]},{lon[k]}' # this is concatinating the lat and lon of each bikeStation which will be fed into the Fouraquare API.

    fsq_params = {
        "categories": '13003', # searching for Bars here
        "ll": ll,
        "radius": 1000, # bars withing 1000m or (1km) of each bike station
        "limit": 50 # max limit of results (max limit allowed anyway..)
    }

    response = requests.request("GET", fsq_url, params=fsq_params, headers=fsq_headers)

    data = response.json()


    num_of_bars.append(len(data['results'])) # appending number of bars (within a 1km distance) for every bike station to the above created list

    # Appending the bars' distance, more specifically the one with the closest or minimum distance for every bike station.
    # Error handling incase ['distance'] may not exist.
    try:
        min_bars_dist.append(pd.json_normalize(data['results'])['distance'].min()) # appending the minimum distance found of all distances within 1km of bike station
    except KeyError:
        min_bars_dist.append(None) # Will fill with 0 incase distance does not exist.

    # Like we peformed above for 'bars' instead now we're looking at 'libraries'.
    fsq_params = {
        "categories": '12080', # library category id/code
        "ll": ll, # feeding in each bike station lat/lon.
        "radius": 1000, # libraries within 1000m or (1km) of each bike station
        "limit": 50 # max limit of results
    }

    response = requests.request("GET", fsq_url, params=fsq_params, headers=fsq_headers)

    data = response.json()


    num_of_libs.append(len(data['results']))  # appending number of library's for every bike station (within a 1km distance) to the above created list
    
    # Error handling incase ['distance'] may not exist.
    try:
        min_libs_dist.append(pd.json_normalize(data['results'])['distance'].min())
    except KeyError:
        min_libs_dist.append(None)


In [25]:
# printing all possible info given by JSON but we wont be needing most of it - nonetheless it gives an idea of what it all looks like,
# atleast a single row.
pd.json_normalize(data['results']) 


,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region
0,4c23e416f7ced13a88f6216d,"[{'id': 12080, 'name': 'Library', 'short_name'...",[],VeryLikelyOpen,500,/v3/places/4c23e416f7ced13a88f6216d,Etobicoke Public Library,America/Toronto,43.598233,-79.509258,43.598233,-79.509258,110 11th St,CA,Lakeshore W & Morrison,"110 11th St (Lakeshore W & Morrison), Toronto ...",Toronto,M8V 3G5,ON


In [33]:
# Using the above created, here we're building out the DataFrame - 'bike_station#', 'latitude', 'longitude' and 'free_bikes' come from the BikeShareToronto.csv but 'num_of_bars'
# 'min_bar_dist', 'num_of_libs' and 'min_libs_dist' were created from the foursquare API.

df_bikes = cityBikes.iloc[:,:] # selecting the desired rows and all columns from cityBikes

# Creating new columns for number of bars and libraries within 1km radius and the bar and lib with minimum distance to bike station
df_bikes['num_of_bars'] = num_of_bars
df_bikes['min_bar_dist'] = min_bars_dist
df_bikes['num_of_libs'] = num_of_libs
df_bikes['min_libs_dist'] = min_libs_dist

# renaming column' 'Unnamed: 0' to 'bike_station#'.
df_bikes = df_bikes.rename(columns={'Unnamed: 0': 'bike_station#'})

# Exporting csv
df_bikes.to_csv('fsq_BarLibs_BikeRadius.csv')

df_bikes


,bike_station#,latitude,longitude,free_bikes,num_of_bars,min_bar_dist,num_of_libs,min_libs_dist
0,0,43.665269,-79.319796,18,11,608.0,2,810.0
1,1,43.671420,-79.445947,2,19,292.0,1,664.0
2,2,43.666224,-79.317693,10,8,413.0,2,766.0
3,3,43.653236,-79.376716,4,50,122.0,9,302.0
4,4,43.663722,-79.380288,23,46,43.0,6,555.0
...,...,...,...,...,...,...,...,...
712,712,43.601915,-79.499419,1,0,NaN,1,885.0
713,713,43.642852,-79.531995,1,8,159.0,0,NaN
714,714,43.650945,-79.379498,0,50,139.0,0,NaN
715,715,43.707356,-79.437231,0,0,NaN,0,NaN
